In [1]:
import pickle
import os
import pandas as pd
import regex as re                  # import the regex module to check valid details                      
from IPython.display import display # import the IPython.display to display the pandas dataframe in a neat tabular form. P.S print(df) doesn't present the dataframe in a tabular form  
from termcolor import colored  

In [ ]:
# contactfile.pkl is the file created where all the previous contacts are being stored
# during the first run the file hasn't been created and a empty dataframe is being created which gets stored in the pickle file as soon as first contact has been created   


if not os.path.exists('contactfile.pkl'):
  file = pd.DataFrame()
  
else:
  with open('contactfile.pkl' , 'rb') as f:
    file =  pickle.load(f) 
      
name_reg_ex = '[a-zA-Z]+\s[a-zA-Z]+\s?[a-zA-Z]?'                                      # regex to validate the name
number_reg_ex = '^\d{10}$'                                                            # regex to validate the phone number 
email_reg_ex = '^[a-zA-Z0-9.!#$%&*+/=?^_`{|}~-]+@[a-zA-Z0-9-]+(?:\.[a-zA-Z0-9-]+)*$'  # regex to validate the email address 
 
user_interface = '''
1. Display your existing contacts 
2. Create a new contact
3. View an entry 
4. Delete a contact
5. Reset(Delete) whole contacts
6. Exit
'''
 
 
 
def user_choice(): 
  '''user_choice function to prompt the user for the choice they would like to execute. if user enters some invalid entry, it prompts the user until a valid entry has been entered'''                                   
  
  print(colored(user_interface , 'green'))
  choice = input("\nChoose among 1/2/3/4/5 or 6 -->")
  
  while choice.isdigit() != True or int(choice) >= 7:            # if user enters other than a digit or when enters a integer greater than 7, it's a invalid entry
   
    print(colored("\nOops! Invalid entry. Please try again." , "white" , "on_red"))
    print(colored(user_interface , 'green'))
    choice = input('Please try again. Choose 1/2/3/4/5 or 6 -->')
  
  return choice
 
 
# all the below defined functions have no use if the file(dataframe) is empty so the empty condition is checked at the beginning which ensures that the file is not empty
 
def displayall():
  ''' function to display the dataframe'''
  
  if file.empty != True:                      
    display(file)  
      
    with open('contactfile.pkl' , 'wb') as f:         # once the dataframe is displayed, it is then pickled for another use
      pickle.dump(file, f)
     
  else:
    print(colored("\nNo any contacts in the dataframe. Please add a contact !" , "white" , "on_red"))
    
def add():
  '''function to add the new contacts'''
  

  contact_name = None
  contact_name = input("\nEnter the name: ")
  name_valid_or_not = re.match(name_reg_ex, contact_name)         # check if the entered name matched with regex or not
  
  while name_valid_or_not == None:                                # if the condition is not matched, the user is prompted for a valid name till he enters a valid one
    
    print(colored("\nOops! That's not a valid name. Try again" , "white" , "on_red"))                                   # error message
    print(colored("\nUser should have first and last name (middle name optional) and have space in between" , "blue"))  # user_guidance
    contact_name = input("\nEnter name again: ")                  # prompting the user for the name again
    name_valid_or_not = re.match(name_reg_ex, contact_name)       # check the regex condition again
  

  phone_number = None
  phone_number = input("Enter the phone number: ")
  phone_valid_or_not = re.match(number_reg_ex, phone_number)
  
  while phone_valid_or_not == None:
    
    print(colored("\nOops! That's not a valid number. Try again" , "white" , "on_red"))
    print(colored("Phone number is of 10 digits" , "blue"))
    phone_number = input("\nEnter phone number again: ")
    phone_valid_or_not = re.match(number_reg_ex, phone_number)
   

  email_address = None
  email_address = input("Enter email address: ") 
  email_valid_or_not = re.match(email_reg_ex, email_address )
  
  while email_valid_or_not == None:
    
    print(colored("\nOops! That's not a valid email. Try again", "white" , "on_red"))
    print(colored("Email format: something@domain_name.com" , "blue"))
    email_address = input("\nEnter email address again: ")
    email_valid_or_not = re.match(email_reg_ex, email_address)
  
  return (contact_name, phone_number, email_address)

def search():
  ''' function to search the contacts'''
  
  if file.empty != True:
    search = input('\nEnter name/phone number/email which you wish to view: ') 
    
    if search in file.values:                                         
      print(colored('\nContact found' , 'blue'))
      display(file.loc[file.isin([search]).any(axis=1)])               # it gets the index for rows by matching search term (could be name/phone/email) in all columns and at last display the rows
    else:
      print(colored('\nNo any records found !' , 'white' , 'on_red'))
  
  else:
    print(colored("\nNo any contacts in the dataframe. Please add a contact !" , "white" , "on_red"))
 
def delete_contact():
  '''function to delete a contact'''
  
  new_deleted_df = None
  to_delete = None
  
  if file.empty != True:
    to_delete = input('\nEnter the name/email/phone number of person whose details you wish to delete: ')   # prompting user for the the details of the person whose contact they want to delete
    
    if to_delete in file.values:
      print(colored('\nContact found.', 'blue'))
      display(file.loc[file.isin([to_delete]).any(axis=1)])                                                 # it gets the index for rows by matching delete term (could be name/phone/email) in all columns and at last display the rows               
      indexes = file.loc[file.isin([to_delete]).any(axis=1)].index.tolist()                                 # to get list of the row index for the above condition
      
      confirm_index = input('\nPlease type in the index you wish to delete: ')                              
      while confirm_index not in str(indexes):
        print(colored('\nYou have to enter the row index of the above dislayed dataframe to proceed the delete choice.' , 'white' , 'on_red'))
        confirm_index = input('\nPlease type in the correct index: ')
      
      new_deleted_df = file.drop(labels = int(confirm_index) , axis=0).reset_index(drop=True)               # a new dataframe is returned by dropping the required row and is then stored in a variable called new_deleted_df
      
    else:
      print(colored("\nNo such contact found !" , "white" , "on_red"))
  
  else:
    print(colored("\nNo any contacts in the dataframe. Please add a contact !" , "white" , "on_red"))
    
  return (new_deleted_df, to_delete)
 
def deleteall():
  ''' function to erase all contacts'''
  
  global file
  if file.empty != True:
    file = pd.DataFrame()                    # the dataframe is emptied             
    print(colored('The contacts have been erased !' , 'blue'))
    os.remove('contactfile.pkl')

  
  else:
    print(colored("\nNo any contacts in the dataframe. Please add a contact !" , "white" , "on_red"))

 
choice= ''                                # initalizing the value of choice which stores input entered by the user
while choice != '7':
  choice = user_choice()
  if choice == '1': 
    displayall()
  
  elif choice == '2':
    details = add()
    contact_name, phone_number, email_address = details[0], details[1], details[2]
    if file.empty != True:
      if phone_number in file['Phone Number'].values or email_address in file['Email'].values:           # if the phone number or the email has been repeated, it is not accepted in the contact book
        print(colored('\nThis contact present already. Please try again.' , 'white' , 'on_red'))
        continue 
      else:
        pass
      
    else:
      pass 
               
 
    contact_dict = {'Name': [contact_name] ,                                    # a dictionary is created that stores details of person  
                    'Phone Number': [phone_number],
                    'Email': [email_address] }
    
    print(colored('\nContact added succesfully ! ' , 'blue'))
    
    
    df0 = pd.DataFrame(contact_dict)                                            # the dataframe from the contact_dict is created 
    file = pd.concat([file, df0] , ignore_index = True)                         # the dataframe is then appended to the previous dataframe and so on 
    displayall()
    
   
  elif choice == '3':
    search()
  
  elif choice == '4':
        
    contacts_deleted = delete_contact()    
    new_deleted_dataframe = contacts_deleted[0]
    to_be_deleted = contacts_deleted[1]
     
    if file.empty != True:
      if to_be_deleted not in file.values:
        continue
      
      else:
        file = new_deleted_dataframe
        print(colored('\nContact deleted succesfully ! ' , 'blue'))
        displayall()
    
    else:
      pass
    
  
  elif choice == '5':
    deleteall()
    
  else:
    print(colored("Exiting....Exited" , "magenta"))


1. Display your existing contacts 
2. Create a new contact
3. View an entry 
4. Delete a contact
5. Reset(Delete) whole contacts
6. Exit


Choose among 1/2/3/4/5 or 6 -->1

No any contacts in the dataframe. Please add a contact !

1. Display your existing contacts 
2. Create a new contact
3. View an entry 
4. Delete a contact
5. Reset(Delete) whole contacts
6. Exit

